In [1]:
CUDA_VISIBLE_DEVICES=""

In [2]:
import jax
import jax.numpy as jnp
from jax import grad, jit, vmap
from jax import random
from jax import device_put
import numpy as np
from numpy import random

jax.config.update('jax_platform_name', 'cpu')

In [3]:
jax.devices()

[CpuDevice(id=0)]

In [4]:
@jit
#@jax.value_and_grad
def TV(x):
    """Computes the total variation norm and its gradient. From jcjohnson/cnn-vis."""
    eps = jnp.float32(1e-8)
    x_diff = x - jnp.roll(x, -1, axis=1)
    y_diff = x - jnp.roll(x, -1, axis=0)
    z_diff = x - jnp.roll(x, -1, axis=2)

    k = x_diff**2 + y_diff**2 + z_diff**2 + eps
    norm = jnp.sum(jnp.sqrt(k))
    return norm

grad_TV = grad(TV)

In [5]:
arr = np.array(random.rand(300, 300, 300), dtype=np.float32)
#arr_gpu = np.array(random.rand(300, 300, 300), dtype=np.float32)
#arr_gpu = device_put(arr_gpu)

In [12]:
%timeit TV(arr).block_until_ready()

55.2 ms ± 180 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
%timeit grad_TV(arr).block_until_ready()

445 ms ± 4.74 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%timeit TV(arr).block_until_ready();

54.7 ms ± 138 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
%timeit grad_TV(arr_gpu)[1].block_until_ready();

TypeError: Gradient only defined for scalar-output functions. Output was (DeviceArray(17671286., dtype=float32), DeviceArray([[[ 0.96534854, -3.4273279 , -1.4476018 , ...,  1.0855145 ,
               -2.0105066 ,  2.2322562 ],
              [ 2.7970405 , -2.5609775 ,  2.7353256 , ...,  1.0619489 ,
                0.37275392,  2.3334022 ],
              [-1.9987123 ,  3.177341  ,  1.9881148 , ..., -3.2968678 ,
               -0.0631274 ,  3.221321  ],
              ...,
              [-1.166106  ,  3.5511615 , -3.2676556 , ...,  3.5613794 ,
               -3.0508475 , -3.0942762 ],
              [-2.4430842 , -2.0590582 , -2.1686807 , ..., -1.0712436 ,
                2.6748548 ,  2.767263  ],
              [ 3.0198536 ,  0.63562846, -1.8635169 , ...,  3.8508499 ,
               -4.119897  , -0.6722129 ]],

             [[-0.647869  ,  0.9406335 ,  2.7573462 , ..., -0.54172194,
               -2.4327872 , -0.86857235],
              [ 2.5939012 , -2.079701  , -2.8692634 , ..., -4.126902

In [22]:
%timeit TV(arr_gpu).block_until_ready();

895 µs ± 2.47 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [24]:
%timeit grad_TV(arr_gpu).block_until_ready();

10.4 ms ± 16 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
